In [1]:
!pip install gradio


In [2]:
import gradio as gr

In [3]:
!pip install --user tensorflow

In [4]:
 !pip3 show tensorflow

Name: tensorflow
Version: 2.11.0
Summary: TensorFlow is an open source machine learning framework for everyone.
Home-page: https://www.tensorflow.org/
Author: Google Inc.
Author-email: packages@tensorflow.org
License: Apache 2.0
Location: c:\users\bhavani omkarini\anaconda3\lib\site-packages
Requires: tensorflow-intel
Required-by: 


In [5]:
import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.vgg19 import VGG19, preprocess_input
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img

In [6]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, GlobalAveragePooling2D

In [7]:
train_datagen = ImageDataGenerator(preprocessing_function=preprocess_input,
                                   rotation_range=20,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True)

val_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

test_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

train_generator = train_datagen.flow_from_directory('OCT2017\\train',
                                                    target_size=(224, 224),
                                                    batch_size=32,
                                                    class_mode='categorical')

val_generator = val_datagen.flow_from_directory('OCT2017\\val',
                                                target_size=(224, 224),
                                                batch_size=32,
                                                class_mode='categorical')

test_generator = test_datagen.flow_from_directory('OCT2017\\test',
                                                  target_size=(224, 224),
                                                  batch_size=1,
                                                  class_mode='categorical',
                                                  shuffle=False)


Found 83484 images belonging to 4 classes.
Found 32 images belonging to 4 classes.
Found 968 images belonging to 4 classes.


In [8]:
from tensorflow.keras.optimizers import SGD
base_model = VGG19(weights='imagenet', include_top=False)

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='elu')(x)
predictions = Dense(4, activation='softmax')(x)

model = tf.keras.Model(inputs=base_model.input, outputs=predictions)

for layer in base_model.layers:
    layer.trainable = False

model.compile(optimizer='SGD', loss='categorical_crossentropy', metrics=['accuracy'])

In [9]:
model.fit(train_generator, epochs=1, validation_data=val_generator)

2609/2609 [==============================] - 23429s 9s/step - loss: 0.5737 - accuracy: 0.8022 - val_loss: 0.3393 - val_accuracy: 0.8750


In [45]:
class_names = os.listdir('OCT2017\\train')

In [48]:
def predict_image(img):
    img_3d=img.reshape(-1,180,180,3)
    prediction = model.predict(img_3d)[0]
    return{class_names[i]:float(prediction[i]) for i in range(4)}

In [51]:
image = gr.inputs.Image(shape=(180,180))
label = gr.outputs.Label(num_top_classes = 4)
gr.Interface(fn=predict_image, inputs=image, outputs=label, capture_session=True).launch(debug='True')


C:\Users\Bhavani Omkarini\anaconda3\lib\site-packages\gradio\inputs.py:257: UserWarning: Usage of gradio.inputs is deprecated, and will not be supported in the future, please import your component from gradio.components
  warnings.warn(
C:\Users\Bhavani Omkarini\anaconda3\lib\site-packages\gradio\deprecation.py:40: UserWarning: `optional` parameter is deprecated, and it has no effect
  warnings.warn(value)
C:\Users\Bhavani Omkarini\anaconda3\lib\site-packages\gradio\outputs.py:197: UserWarning: Usage of gradio.outputs is deprecated, and will not be supported in the future, please import your components from gradio.components
  warnings.warn(
C:\Users\Bhavani Omkarini\anaconda3\lib\site-packages\gradio\deprecation.py:40: UserWarning: The 'type' parameter has been deprecated. Use the Number component instead.
  warnings.warn(value)
C:\Users\Bhavani Omkarini\anaconda3\lib\site-packages\gradio\deprecation.py:40: UserWarning: `capture_session` parameter is deprecated, and it has no effect
 

Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


1/1 [==============================] - 0s 175ms/step
Keyboard interruption in main thread... closing server.
